# Class 9 - 13.5.18

# Testing and Test-Driven Development

In [ ]:
obj = CompareSeries(4)
res = obj.gen_fib_arr()


## Introduction

We've already met one widely adopted programming technique which isn't used as much in the academy - object-oriented programming. Another such technique, ubiquitously used wherever code is written _outside_ the academy, is testing. In my opinion, this is a crucial topic in software development that isn't treated with the respect it should in the academy.

Tests are (usually) short pieces of code designed to assert that a small portion of your program does what you intend it to do. 

For example, if I have a class designed to perform calculations on some DataFrame that was created somewhere else, perhaps by adding some of its columns together, averaging them out and displaying the result, then I wish for my code to be correct and deterministic, in a sense that a single, defined input will always give the same correct output.

This isn't trivial even for the most basic functions in Python. Let's try to build a "wrapper" over `np.tile`:

In [1]:
# A reminer of np.tile()
import numpy as np


arr = np.array([1, 2, 3])
tiled = np.tile(arr, (2, 3))  # repeat it twice in the row dimension (axis=0), 
                              # and three times in the column dimension
tiled

array([[1, 2, 3, 1, 2, 3, 1, 2, 3],
       [1, 2, 3, 1, 2, 3, 1, 2, 3]])

In [2]:
# Our own implementation

class Tiler:
    """ Tile any number of objects on their first axis """
    def __init__(self, *args):
        self.to_tile = args

    def tile(self, reps=(1,)):
        self.tiled = np.tile(self.to_tile, reps=reps)
        return self.tiled

The above class can tile as many array inputs as you wish, and currently it's just a fancy wrapper over `np.tile()`. 


In [3]:
tiled = Tiler([1, 2, 3], [4, 5, 6]).tile()
print(tiled)

print('------')
tiled_again = Tiler([1, 2, 3], [4, 5, 6]).tile((2, 2))
print(tiled_again)

[[1 2 3]
 [4 5 6]]
------
[[1 2 3 1 2 3]
 [4 5 6 4 5 6]
 [1 2 3 1 2 3]
 [4 5 6 4 5 6]]


It's a very simple implementation, with the only "new" thing being the `*args` argument to the `__init__` function. `*args` means: Pack into a tuple called `args` all non-keyword arguments that the function receives. For example:

In [4]:
def f(a, b, *args):
    print(f"a is {a}")
    print(f"b is {b}")
    print(f"The rest of the data received is {args}")
    
f(1, 2, 3, 4, 'a')

a is 1
b is 2
The rest of the data received is (3, 4, 'a')


We use `*args` when we wish to write a function with a non-constant number of arguments. The "magic" is done with the star (`*`), the `args` is just a convention, it's not a keyword. The star knows to bind the inputs into a tuple. In our case, we wish the `Tiler` class to tile as many arrays as the user wishes. This is most easily done with the `*args` parameter. Like `*args`, Python also has `**kwargs` that binds the keyword argument inputs into a dictionary.

So now that we understand the class completely - does it work?

It does work, but it's not _guaranteed_ to work. Some issues are clear and can be solved immediately, like some basic type-checking that we should implement:

In [5]:
class Tiler:
    """ 
    Tile any number of objects on their first axis.
    Returns: np.ndarray tiled across the first dimension.
    Errors: TypeError if supplied with invalid iterables
    """
    def __init__(self, *args):
        _to_tile = self._verify_inp(args)
        self.to_tile = _to_tile

    def tile(self, reps=(1,)):
        self.tiled = np.tile(self.to_tile, reps=reps)
        return self.tiled
    
    def _verify_inp(self, it):
        to_tile = []
        for item in it:
            if not isinstance(item, (list, np.ndarray)):
                raise TypeError(f'Item {item} not a valid iterable to tile.')
            to_tile.append(item)
        return to_tile

Now the class specifies its return type and what happens in case of an error. A good application will know to wrap the use of the class in a `try... except` block and catch any `TypeError`s:

In [6]:
# This works:
try:
    tiled = Tiler([1, 2, 3], [4, 5, 6]).tile()
except TypeError as e:
    print(f"TypeError: {e}")
print(tiled)

# This shouldn't
print("--------")
try:
    tiled_wrong = Tiler([1, 2, 3], {4, 5, 6}).tile()  # input is a set, not a list
except TypeError as e:
    print(f"TypeError: {e}")

[[1 2 3]
 [4 5 6]]
--------
TypeError: Item {4, 5, 6} not a valid iterable to tile.


Are we covered? Is this class no longer vulnerable to any type of non-legitimate input? Of course it is vulnerable, and some of us might already see a few corner cases:

In [7]:
try:
    tiled = Tiler([], np.array([1, 2, 3])).tile()
except TypeError as e:
    print(f"TypeError: {e}")

tiled  # what is that?

array([list([]), array([1, 2, 3])], dtype=object)

This array of lists and arrays is certainly not what we had in mind when tiling an empty list an a simple array. We can think what exactly we expect this function to do, but this output will certainly not be on the list of possible outputs.

I really don't have to convince you that software, even the most basic application, has bugs. But _thinking_ that we solved our bugs by the insertion of methods like `self._verify_inp` doesn't mean we actually solved them.

To this end we write tests for our program. You've already seen unit tests in your homework assignments. You were asked to make sure that your solution passes all tests before submitting. This exemplifies a way through which we can be more certain that our program does what we thought it was doing.

Tests are important to us for two reasons. The first one is that even simple programs are more complicated than we think. The `Tiler` class is a good example, but you might imagine that as soon as we add interfaces between classes, methods and functions, things might get a bit messy. For example, in the aviation industry, for each line of code a software has, you may find about 8 lines dedicated to testing it.

Moreover, when we deal with user input - data files, parameters for script - we should expect the unexpected, even if the main user is ourselves. Our future self in a few months will probably not remember the type of every parameter it has to enter.

The second reason is Python's dynamic nature, or _duck-typing._ If you want to enforce a function to only accept inputs of a single type, _you_ must be the one writing these assertions, either outside or inside the function. For example, a function that adds two numbers needs a `isinstance(value, (int, float))` somewhere near its top to avoid these mistakes. Statically-typed languages, like C, define a type for each variable. A function adding two integers simply cannot accept a non-integer input. 

Python's dynamic nature is a blessing on many occasions, but it can sometimes be a real pain. This nature is the second important reason to write tests to our code. Many cases that in other programming languages would've resulted in a simple `TypeError`, can cause major bugs in Python due to wrong input types.

## Solution

### Test-Driven Development

Test-Driven Development, or TDD, is a very popular way to solve the problems we described. In TDD we reverse the process of writing code. We first write a test to the function we wish to write. It fails - because the function doesn't exist yet - and then we write then function until we're passing our test. We then add more tests to make sure that the function works properly.

Let's assume I wish to write a function that adds two positive integers. First I'll write a few basic tests for the function, and then I'll try to run them.

In [8]:
# Test functions always start with "test_..."
def test_basic_addition():
    first = 2
    second = 4
    result = 6
    return intadd(first, second) == result
    
def test_negative_inp1():
    try:
        result = intadd(-1, 1)
    except TypeError:
        return True
    else:
        return False

def test_negative_inp2():
    try:
        result = intadd(1, -1)
    except TypeError:
        return True
    else:
        return False

After the tests are written (and are failing, since the fucntion `intadd` doesn't exist, we need to write the function so that it will comply with the tests we have.

In [9]:
def intadd(num1, num2):
    """ Non-negative integer addition """
    if (num1 < 0) or (num2 < 0):
        raise TypeError('Input must be positive.')
    return num1 + num2

In [10]:
print(test_basic_addition())
print(test_negative_inp1())
print(test_negative_inp2())

True
True
True


Our function works! See how each test deals with a unique sub-case of failure? It's much easier to debug and answer "what went wrong?" when you tests are so precise. A good tests consists of a couple of lines of initialization, a couple of lines that call the tested function, and the `assert`ing line.

During the time it took me to write the implementation, I thought of a different edge case - floating point numbers input. Now I should write tests that will check this input type, they will fail - and then I can refactor my `intadd` function to deal with these cases.

In [11]:
def test_float_inp():
    """ A shorter version - check both inputs in a single test function """
    try:
        result = intadd(1., 2)
        result = intadd(2, 1.)
    except TypeError:
        return True
    else:
        return False

In [12]:
print(test_float_inp())  # False!

False


In [13]:
def intadd(num1, num2):
    """ Non-negative integer addition """
    if (num1 < 0) or (num2 < 0):
        raise TypeError('Input must be positive.')
    if isinstance(num1, float) or isinstance(num2, float):
        raise TypeError('Input must be integer.')
    return num1 + num2

In [14]:
print(test_float_inp())  # True!

True


And the process continues - every time I think of more edge cases, I write failing tests and then refactor my own function.

Another important thing to note here is the way TDD forces me to "design the API" of my function. As you can see, when I wrote the test I had to think of the output I'll receive when I enter wrong inputs. In this case I thought it was most appropriate for the function to return an TypeError - an exception that if not caught can terminate the execution of my application. This appeared to be the right choice here, but sometimes we can define a different result for faulty inputs. We could've returned `-1` if the input was invalid, for example.

With TDD, most people don't actually write the first tests before they write the function. TDD purists might insist on that, but for all intents and purposes, if you keep the development of the tests very closely tied to the development of the function itself - you're doing TDD.

### Actual usage - `pytest`

As you might have guessed, the Python ecosystem offers tools to automate the testing process. The standard library comes with a `unittest` module, and another famous one is `nosetests`. But the most popular (and advanced) library as of early 2018 is [`pytest`](https://docs.pytest.org/en/latest/), and that will be the our library of choice.

In the `tests_demo` folder you can see how one structures the tests a project.

After creating this file structure, you just call `pytest` in the command line after you `cd`ed into the folder containing the project - and all tests are run for you.

`pytest` has some advanced features. In the tests inside `tests_demo` you could already see the `parametrize` decorator, used to call the same test with several different inputs. `pytest` is smart enough to tell us which input out of the ones we entered caused the exception. In addition, you can also mark some tests as "expected to fail". Finally, It can also create automatic tests for you. [Here](https://docs.pytest.org/en/latest/parametrize.html#basic-pytest-generate-tests-example) you'll find the formal docs, and [here](https://hackebrot.github.io/pytest-tricks/create_tests_via_parametrization/) you can find a clear blog post explaining it.

#### A few extra points:

1. Tests should be as concise as possible. Their execution time should be minimal.
2. Run your test suit as often as possible. Minimal frequency is before each commit you make.
3. You should try hard to translate bugs into tests. These might be the most important tests you'll write.
4. Test names are long. It's fine - it's because we don't use docstrings for tests.
5. You can configure PyCharm to work with `pytest` in _File - Settings - Tools - Python Integrated Tools - Default test runner: py.test._

### Integration Testing

Unit tests repeatedly tests functions or methods your write under different inputs, and are the backbone of any reliable test suit. However, unit tests are not enough, since they don't check the interface between the different functions and classes in your application.

Integration tests are larger, heavier tests that take at least two components, or units, of your application, and makes sure that they interact well with each other.

Obviously, if we start taking each two consecutive functions and write an integration test for that pair, and then continue with all three consecutive functions and write these integration tests, and so on - we'll never finish writing the damn application. That's why integration testing is used at crucial junctions of our application, between major classes for example.

## Exercise

Write a class that returns the difference between the Fibonacci series and the prime numbers series, up to some _n._ The output should be an array of numbers, n-items long, and a plot to accompany it. As an example, for $n=3$, I expect the array to be `np.array([-2, -2, -4])`. Write the class in a test-driven development style.

A reminder, the Fibonacci series is a series of numbers starting from (0, 1), with its next element being the sum of the previous two numbers: 0, 1, 1, 2, 3, 5, ... Prime numbers start from 2 and are only divisible by themselves and 1 without a remainder.

You decide on the class' interface and the details of implementation. You may use `numpy`, and I insist that you write at least 5 unit tests and 1 integration test for this class.

Don't try to implement things in a performant way, with fancy algorithms. The focus here is the unit tests and test-driven development.

### Exercise solutions are in the directory `fib_ser`